In [14]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os

In [15]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [16]:
def calculate_angle(a,b,c):
    # a is first angle, b is second angle, c is 3rd angle
    a = np.array(a)
    b = np.array(b)
    c = np.array(b)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)
    if (angle > 180):
        return 360-angle;
    return angle

In [17]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) # confidence metric c, dont transform that
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #.circle creates circle, 4 is size, 255 is color green, -1 is fill circle

In [18]:
# nose 0, left eye 1, right eye 2, left ear 3, right ear 4, 
# left shoulder 5, right shoulder 6, left elbow 7, right elbow 8, 
# left wrist 9, right wrist 10, left hip 11, right hip 12, left knee 13, right knee 14, left ankle 15, right ankle 16
# elbow, shoulder, hip, ankle

In [19]:
EDGES = {
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [20]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1-5]
        y2, x2, c2 = shaped[p2-5]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [23]:
def test():
    l_one = [] # l = the last frame recorded
    l_l_one = [] # the last last frame recorded
    went_down = True
    squat_count = 0
    frame_idx = 0 
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        # take frame from open cv video capture
        ret, frame = cap.read()
        img = frame.copy()
        #start of taking coordinates
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0), 192, 192)
        input_image = tf.cast(img, dtype=tf.float32)
        
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        kyps_w_scores = interpreter.get_tensor(output_details[0]['index'])
        # end of taking coordinates

        # 5,6 shoulder; 7,8 elbow; 11,12 hip; 13,14 knee; 15,16 ankle
        # sectioning off all scores were interested in
        wanted_scores = kyps_w_scores[0][0][5:]
        # all angles for hands - 1st left shoulder, hip, knee
        back_angle = [calculate_angle(wanted_scores[0], wanted_scores[6], wanted_scores[8])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[9])]
        # all angles for legs - 1st left hip, knee, ankle
        knee_angle = [calculate_angle(wanted_scores[6], wanted_scores[8], wanted_scores[10])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[11])]
        
        """# setting the 2 arrays to the first 2 frames
        if (frame_idx < 2):
            l_l_one = l_one
            l_one = [back_angle, knee_angle]
            frame_idx += 1
            continue
            
        # went dowm for if the hands went down (not properly checking)
        # if hands alr went down
        # if last last underarm < last underarm and last underarm > current (then it's going back up)
        # if up angle > 130 and bottom < 30
        if (went_down and l_one[0][0] > 130 and l_l_one[0][0] < l_one[0][0]  and l_one[0][0] > underarm_angle[0]):
            print(underarm_angle[0])
            jj_count += 1
            went_down = False
            # print(jj_count)
        elif (went_down == False and l_one[0][0] < 30 and l_l_one[0][0] > l_one[0][0]  and l_one[0][0] < underarm_angle[0]):
            print(underarm_angle[0])
            went_down = True
        
        l_l_one = l_one
        l_one = [underarm_angle, leg_angle]
        full_text_for_squats = "Squats: " + str(squat_count)"""
        print(back_angle)
        print(knee_angle)
        if (went_down and ((back_angle[0] > 130 and knee_angle[0] > 130) or (back_angle[1] > 130 and knee_angle[1] > 130))):
            squat_count += 1
            went_down = False
            print(squat_count)
        elif (went_down == False and ((back_angle[0] < 60 and knee_angle[0] <= 95) or (back_angle[1] < 60 and knee_angle[1] <= 95))):
            went_down = True
        
        full_text_for_squat = "Squats: " + str(squat_count)
        
        cv2.putText(frame,  
                full_text_for_squat,  
                (50, 50),  
                cv2.FONT_HERSHEY_SIMPLEX, 1,  
                (0, 0, 0),  
                3,  
                cv2.LINE_4)
        draw_keypoints(frame, wanted_scores, 0.4)
        draw_connections(frame, wanted_scores, EDGES, 0.4)
        cv2.imshow('Squats', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    # h_avg_up = h_angle_up / jj_count
    # h_avg_down = h_angle_down / jj_count
    # l_avg_up = l_angle_up / jj_count
    # l_avg_down = l_angle_down / jj_count
    return #h_avg_up, h_avg_down, l_avg_up, l_avg_down

In [27]:
test()

[33.87682303791446, 177.34664044094646]
[131.43693653843712, 177.34664044094646]
1
[17.704229404171986, 178.9008363262045]
[137.25877122090256, 178.9008363262045]
[20.03554372262659, 121.44932850004591]
[138.94834145551027, 121.44932850004591]
[26.683459049353793, 168.05169554666458]
[144.1267996389172, 168.05169554666458]
[30.830456215251225, 179.7959326169347]
[139.85697518117541, 179.7959326169347]
[14.39702960993701, 175.91265588508062]
[156.27476919102676, 175.91265588508062]
[14.28598524818459, 7.853015120864038]
[142.11473519245814, 7.853015120864038]
[54.32695924316083, 178.96688425547853]
[156.56222453243592, 178.96688425547853]
2
[145.01841253317065, 179.2837913725888]
[159.54925750177927, 179.2837913725888]
[173.91608230765917, 175.0156608017612]
[8.854542834335932, 175.0156608017612]
[165.32561678475074, 163.8193824878445]
[9.021057723658966, 163.8193824878445]
[165.113184241185, 167.05652332421536]
[1.2244737324392791, 167.05652332421536]
[156.23321432011642, 156.092881253